In [1]:
# Inspect all event types and data (mimicking api.py behavior)
from dotenv import load_dotenv
from pprint import pprint
import json

load_dotenv()
print("Environment loaded")

Environment loaded


In [2]:
# Create a dummy tool that returns mock artifacts
from langchain_core.tools import tool
from langchain_core.messages import ToolMessage
from langgraph.types import Command
from typing_extensions import Annotated
from langchain_core.tools import InjectedToolCallId

@tool
def dummy_tool(
    query: Annotated[str, "A query string"],
    tool_call_id: Annotated[str, InjectedToolCallId]
) -> Command:
    """A dummy tool that returns mock artifacts"""
    
    # Simulate artifacts like code_sandbox returns
    mock_artifacts = [
        {
            "name": "test_image.png",
            "mime": "image/png",
            "url": "/api/artifacts/123?token=abc",
            "size": 12345
        },
        {
            "name": "data.csv",
            "mime": "text/csv",
            "url": "/api/artifacts/456?token=def",
            "size": 67890
        }
    ]
    
    # Return ToolMessage wrapped in Command (like code_sandbox does)
    tool_msg = ToolMessage(
        content=f"Processed query: {query}",
        artifact=mock_artifacts,
        tool_call_id=tool_call_id
    )
    
    return Command(update={"messages": [tool_msg]})

print("Dummy tool created")

Dummy tool created


In [3]:
# Create a simple graph with just the dummy tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

# Create LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0.7)

# Create agent with just the dummy tool
memory = MemorySaver()
graph = create_react_agent(llm, tools=[dummy_tool], checkpointer=memory)

print("Graph created with dummy tool")

Graph created with dummy tool


In [4]:
# Stream ALL events and show what would be sent to frontend
user_text = "Use the dummy tool to search for 'test query'"
state = {"messages": [{"role": "user", "content": user_text}]}
config = {"configurable": {"thread_id": "test-123"}}

print("=" * 80)
print("STREAMING ALL EVENTS (mimicking api.py)")
print("=" * 80)

event_count = {}

async for event in graph.astream_events(state, config, version="v2"):
    event_type = event.get("event")
    name = event.get("name")
    data = event.get("data", {})
    metadata = event.get("metadata", {})
    
    # Count event types
    event_count[event_type] = event_count.get(event_type, 0) + 1
    
    # Filter to relevant events (like api.py does)
    if event_type in ["on_chat_model_stream", "on_tool_start", "on_tool_end"]:
        print(f"\n{'='*80}")
        print(f"EVENT: {event_type}")
        print(f"NAME: {name}")
        print(f"METADATA: {metadata}")
        print(f"{'-'*80}")
        
        if event_type == "on_chat_model_stream":
            # Show what chunk would be sent to frontend
            chunk = data.get("chunk")
            if chunk:
                content = getattr(chunk, "content", "")
                print(f"CHUNK CONTENT: {repr(content)}")
                
                # Check for tool calls
                if hasattr(chunk, "tool_call_chunks") and chunk.tool_call_chunks:
                    print(f"TOOL CALL CHUNKS: {chunk.tool_call_chunks}")
        
        elif event_type == "on_tool_start":
            # Show tool input
            tool_input = data.get("input")
            print(f"TOOL INPUT:")
            pprint(tool_input, indent=2)
            
            # This is what would be sent to frontend as SSE
            sse_data = {
                "event": "tool_start",
                "tool_name": name,
                "tool_input": tool_input
            }
            print(f"\nSSE TO FRONTEND:")
            print(json.dumps(sse_data, indent=2))
        
        elif event_type == "on_tool_end":
            # Show raw tool output
            raw_output = data.get("output")
            print(f"RAW OUTPUT TYPE: {type(raw_output).__name__}")
            print(f"RAW OUTPUT:")
            pprint(raw_output)
            
            # Extract ToolMessage (like api.py does)
            artifacts = None
            tool_call_id = None
            content = None
            
            if hasattr(raw_output, "update") and isinstance(raw_output.update, dict):
                msgs = raw_output.update.get("messages", [])
                if msgs:
                    tm = msgs[0]
                    tool_call_id = getattr(tm, "tool_call_id", None)
                    content = getattr(tm, "content", None)
                    artifacts = getattr(tm, "artifact", None)
            
            print(f"\nEXTRACTED:")
            print(f"  tool_call_id: {tool_call_id}")
            print(f"  content: {content}")
            print(f"  artifacts:")
            pprint(artifacts, indent=4)
            
            # This is what would be sent to frontend as SSE
            sse_data = {
                "event": "tool_end",
                "tool_name": name,
                "tool_call_id": tool_call_id,
                "content": content,
                "artifacts": artifacts
            }
            print(f"\nSSE TO FRONTEND:")
            print(json.dumps(sse_data, indent=2, default=str))

print(f"\n\n{'='*80}")
print("EVENT SUMMARY")
print(f"{'='*80}")
for event_type, count in sorted(event_count.items()):
    print(f"{event_type}: {count}")
print(f"\nTotal events: {sum(event_count.values())}")

STREAMING ALL EVENTS (mimicking api.py)

EVENT: on_chat_model_stream
NAME: ChatOpenAI
METADATA: {'thread_id': 'test-123', 'langgraph_step': 1, 'langgraph_node': 'agent', 'langgraph_triggers': ('branch:to:agent',), 'langgraph_path': ('__pregel_pull', 'agent'), 'langgraph_checkpoint_ns': 'agent:0aaed217-d10e-4ea9-98bd-3ee7bff94ebe', 'checkpoint_ns': 'agent:0aaed217-d10e-4ea9-98bd-3ee7bff94ebe', 'ls_provider': 'openai', 'ls_model_name': 'gpt-4o', 'ls_model_type': 'chat', 'ls_temperature': 0.7}
--------------------------------------------------------------------------------
CHUNK CONTENT: ''
TOOL CALL CHUNKS: [{'name': 'dummy_tool', 'args': '', 'id': 'call_l0XtljxikTK9sqgW1U7ISlap', 'index': 0, 'type': 'tool_call_chunk'}]

EVENT: on_chat_model_stream
NAME: ChatOpenAI
METADATA: {'thread_id': 'test-123', 'langgraph_step': 1, 'langgraph_node': 'agent', 'langgraph_triggers': ('branch:to:agent',), 'langgraph_path': ('__pregel_pull', 'agent'), 'langgraph_checkpoint_ns': 'agent:0aaed217-d10e-4ea9